# 7.01 Lesson 1 key concepts
## Reading the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('C:/Users/igrav/Desktop/ironhack/Lecture/Week_7/Day1/AM/learningSet.csv')

In [ ]:
print(data.shape)
data.head()

Column "TARGET_B" encodes if the customer answered or not the call

In [ ]:
data['TARGET_B'].value_counts()

Column "TARGET_D" provide the donation amount.

In [ ]:
data['TARGET_D'].describe()

In [ ]:
data[data['TARGET_B']>0]['TARGET_D'].describe()

To solve this problem we will first build a classification model to predict who will more likely respond and then for those respondents, we will build a regression model to predict the donation amount.

Then we can use the cost matrix to calculate the total benefit from the donations (remember that the company has to do a mailing champaing and each envelope costs some money).

Some of the challenges with the dataset are as follows:

* Large number of features: The data set has over 450 features. Hence selecting the right features for the model is very critical and at the same time it is not easy as the same traditional ways of removing features is not effective given the large number of features. Apart from feature selection, feature extraction (creating your own features using the existing features) is also not easy in this case.

* Sparsity of the dataset: There are a lot of features with a large number of null values.

* Data imbalance: For developing a classification, there is a huge imbalance in the training dataset with only approximately 5000 values for one category as compared to cover 95,000 instances for the other category.

# 7.01 Lesson 2 key concepts
## Review data cleaning process

There are a lot of columns that have a very high percentage of null values. It is a highly sparse dataset. 
We can decide on a threshold and then remove those variables. There is no rule of thumb to decide on this threshold value. 
Sometimes it can as low as 25%-30%. And sometimes in some data sets you might find that even though there are more than 50% missing values in a column, you might have to include that variable in your analysis. 
A lot of it depends on the business context as well. In this case we will take this threshold to be 25% and then check the definitions of the columns filtered, to see if there is any column that we might want to keep.

In [ ]:
len(data)

In [ ]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

Which columns have NA's?

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

In [ ]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']

In [ ]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

From the list above that includes the columns that have over 25% null values, you discussed with your manager you were told that the following columns are important -> **wealth1, wealth2**
We will remove these variables from the above list 
RDATE3, RAMNT_3 are important but they have too many null values 

# 7.01 Activity 2

Create a function that takes a dataframe as an input and a percentual threshold (default value = 0.25) and returns a list of columns with null values greater than the specified threshold

In [ ]:
# Your code here

In [ ]:
def threshold(df, threshold ):
    nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']
    columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>threshold]
    return list(columns_above_threshold['column_name'])
            

In [ ]:
# Test your function here
threshold(data, 0.25)

## More data cleaning

In [ ]:
data.head()

We can see that there are a lot of columns that have blank spaces which represent no value in this case. 
They were not identified as null values by python as they are empty spaces that are read as character values by 
python. We will replace those values by NaNs and repeat the analysis 

Before we do that we will replace the blank values from the column "MAILCODE" by "A" which would mean the address is okay (pl check the definition of the variable in the description)

In [ ]:
data['MAILCODE'].value_counts()

In [ ]:
data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x.replace(" ", "A"))

In [ ]:
# Now we can replace the rest space characters with np.NaN

In [ ]:
data = data.apply(lambda x: x.replace(" ", np.NaN))

In [ ]:
data.head()

# 7.01 Activity 3

Use the previous function to obtain the columns with null values greater than 0.25 and store those columns in a list called "drop_columns_list"

In [ ]:
# Your code here 
drop_columns_list = threshold(data, 0.25)
drop_columns_list

We would again repeat the same exercise as the last time. We will discuss it with the team, manager, and/or other stakeholders to see which columns we need to retain here. 

Like last time we will keep the following -> **wealth1, wealth2 along with these VETERANS, SOLIH**

Remove those column's names from the list of columns to drop.

In [ ]:
# Your code here. Try to use list comprehension
# list comprehension
my_list = []
for number in range(1,11):
    my_list.append(number)
my_list

# List comprehension syntax
# [value for item in list]
my_list = [ number for number in range(1,11) ]
my_list
my_list = []
for number in range(1,11):
    if ( number%2 == 0 ):
        my_list.append(number)
my_list

# List comprehension syntax with a conditional
# [value for item in list if (condition)]
my_list = [ number for number in range(1,11) if (number%2 == 0)]
my_list
my_list = []
for number in range(1,11):
    if ( number%2 == 0 ):
        my_list.append(number)
    else:
        my_list.append(0)
my_list

# List comprehension syntax with a conditional with an else
# [value if (condition) else other_value for item in list]
my_list = [ number if (number%2 == 0) else 0 for number in range(1,11) ]
my_list

# 7.01 Lesson 4 key concepts
## Separating Target, Numerical Features, and Categorical Features

Since we have a huge number of features, it would be easier to work independently on numerical features and categorical features.

For the target variables, for now we will retain them both together. But later, we will build a classification model first where we would need the column TARGET_B only.

In [ ]:
Y = data[['TARGET_B', 'TARGET_D']]
Y.head()

In [ ]:
numerical = data.select_dtypes(np.number)
numerical = numerical.drop(columns = ['TARGET_B', 'TARGET_D'])
numerical.head()

In [ ]:
numerical.shape

In [ ]:
categorical = data.select_dtypes([object])
categorical.head()

In [ ]:
categorical.shape

## Working with categorical columns

We will work with the categorical features first. Look at the columns one by one. Some of the operations which we will perform are:

- Replace null values with the most occurring categories
- Reduce the number of categories in a column by grouping

It is important to note that some columns are defined by python as categorical/object types. There might be other columns defined as numerical that we want as categorical. We will look them later when we were working on numerical types.

In [ ]:
categorical.isna().sum()

## Categorical Variables

Here we will try to reduce the number of categories. An ideal way would have been to group the states into 
regions. But in this case we will group all the states with counts less than 2500 into one category "other"

In [ ]:
df = pd.DataFrame(categorical['STATE'].value_counts()).reset_index()

df.columns = ['state', 'count']
other_states = list(df[df['count']<2500]['state'])

def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x
    
categorical['STATE'] = list(map(clean_state, categorical['STATE']))

In [ ]:
categorical['STATE'].value_counts()

# 7.01 Lab | Revisiting Machine Learning Case Study

Lab instructions can be found [here](https://github.com/ironhack-labs/lab-revisiting-machine-learning)

# 7.02

We will work on the column DOMAIN. Let's replace the null values with the category that is represented the most and then we will split the data into two columns DOMAIN_A and DOMAIN_B.

- DOMAIN_A will consist of the first character from the DOMAIN
- DOMAIN_B will consist of the second character from the DOMAIN

Then we will drop the original column DOMAIN

In [ ]:
categorical['DOMAIN'].value_counts(dropna=False)

In [ ]:
m = categorical[categorical['DOMAIN'].notna()]['DOMAIN'].mode()[0]
m

In [ ]:
categorical['DOMAIN'] = categorical['DOMAIN'].fillna(m)

Now let's create the new columns 'DOMAIN_A' and 'DOMAIN_B' with the first and second letter of column "DOMAIN" respectivelly

In [ ]:
categorical['DOMAIN_A'] = list(map(lambda x: x[0], categorical['DOMAIN']))
categorical['DOMAIN_B'] = list(map(lambda x: x[1], categorical['DOMAIN']))

Now we drop the original column

In [ ]:
categorical = categorical.drop(columns=['DOMAIN'])

In [ ]:
categorical.head()

### Activity 2.1

Check the instructions [here](https://github.com/ironhack-edu/data_7.01_activities/blob/master/7.01_activity_2.md)

In [ ]:
# Your code here

In [ ]:
drop_columns_list

In [ ]:
#End Activity

MDMAUD_XXX are also like MDMAUD



In [ ]:
categorical['MDMAUD_R'].value_counts()

In [ ]:
drop_columns_list += ['MDMAUD_R', 'MDMAUD_F','MDMAUD_A']

Replacing NULL values

In [ ]:
categorical['CLUSTER'].value_counts(dropna=False)

In [ ]:
m = categorical[categorical['HOMEOWNR'].notna()]['HOMEOWNR'].mode()[0]

In [ ]:
categorical['HOMEOWNR'] = categorical['HOMEOWNR'].fillna(m)

In [ ]:
categorical['GENDER'].value_counts(dropna=False)

In [ ]:
ml =list(categorical[(categorical['GENDER'].isin(['F','M']) == False) & (categorical['GENDER'].notna())]['GENDER'].index)
categorical.iloc[ml,list(categorical.columns).index("GENDER")] = "other"

In [ ]:
categorical['GENDER'].value_counts(dropna=False)

Let's drop the rows with NA's in gender.

In [ ]:
ml = list(categorical[categorical['GENDER'].notna()].index)
categorical = categorical.loc[ml,:]
numerical = numerical.loc[ml,:]

Dealing with redundant information

In [ ]:
categorical['RFA_2'].value_counts()

We will remove all the columns "RFA_2X" except ['RFA_2R','RFA_2A']

In [ ]:
drop_columns_list += [col for col in categorical.columns if (("RFA" in col) and ( col not in ['RFA_2R','RFA_2A']))]

In [ ]:
drop_columns_list

In [ ]:
list(categorical.columns)

In [ ]:
cat_columns_to_drop = [col for col in drop_columns_list if col in list(categorical.columns)]
cat_columns_to_drop

In [ ]:
categorical = categorical.drop(columns=cat_columns_to_drop, axis=1)
categorical.head()

In [ ]:
categorical.isna().sum()

In [ ]:
categorical.isna().sum()/len(categorical)

In [ ]:
[drop_columns_list.remove(col) for col in ["RDATE_3", "RAMNT_3"]]
len(drop_columns_list)
drop_columns_list

In [ ]:
categorical.shape

# Working with numerical columns 

In [ ]:
numerical.head()

In [ ]:
numerical.shape

In [ ]:
df = pd.DataFrame(numerical.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

### Activity 2.2.2

Determine wich numerical columns have a percentage of NA's above 25%.

In [ ]:
# Your code here
def threshold(df, threshold ):
    nulls_percent_df = pd.DataFrame(numerical.isna().sum()/len(numerical)).reset_index()
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']
    columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>threshold]
    return list(columns_above_threshold['column_name'])

In [ ]:
len(threshold(numerical, 0.25))

In [ ]:
np.mean(numerical[numerical['AGE'].notna()]['AGE'])

### Filling NULL values with plots

In [ ]:
#numerical['AGE'] = numerical["AGE"].fillna(np.mean(numerical['AGE']))
fig, ax = plt.subplots(1,3,figsize=(16,5))
sns.histplot(numerical['AGE'], ax = ax[0])
sns.histplot(numerical["AGE"].fillna(np.mean(numerical['AGE'])), ax = ax[1])
sns.histplot(numerical['AGE'].fillna(np.median(numerical['AGE'])),ax = ax[2])
plt.show()
#probably would be a bit better to use the median here to impute
numerical['AGE'] = numerical["AGE"].fillna(np.median(numerical['AGE']))

In [ ]:
#sns.distplot(numerical['INCOME'])  # this will not work as there are NaNs in the column
sns.histplot(numerical[numerical['INCOME'].notna()]['INCOME']) 
plt.show()

In [ ]:
# looks like the variable is actually categorical. We can verify it by using value_counts()
print(numerical['INCOME'].value_counts())
numerical['INCOME'] = numerical['INCOME'].astype('object')

In [ ]:
m = numerical['INCOME'].mode()[0]
numerical['INCOME'] = numerical['INCOME'].fillna(m) # Replacing the null values with the most represented categoty

In [ ]:
sns.histplot(numerical[numerical['CLUSTER2'].isna()==False]['CLUSTER2'],kde=True) 
plt.show()

In [ ]:
numerical['CLUSTER2'] = numerical['CLUSTER2'].fillna(np.ceil(np.mean(numerical['CLUSTER2'])))

###Activity 2.3

Drop all the numerical columns containing "ADATE_. We are assuming that the date when the previous mail was done is not significant in the respondents decision to give donation.

In [ ]:
# Your code here. Try to use list comprehension
drop_columns_list = [col for col in numerical.columns if ("ADATE_" in col)]
drop_columns_list

In [ ]:
# Drop the columns
numerical.drop(drop_columns_list, axis =1)


#Lab 2

Lab instructions can be found [here](https://github.com/ironhack-labs/lab-feature-engineering)

## Check for null values in the numerical columns.

In [ ]:
numerical.isna().sum().sort_values().tail(20)

## Use appropriate methods to clean the columns GEOCODE2, WEALTH1, ADI, DMA,and MSA.

## GEOCODE2

In [ ]:
data['GEOCODE2'].isna().sum()

In [ ]:
fig, ax = plt.subplots(1,figsize=(16,5))
sns.histplot(data['GEOCODE2'])
plt.show()

In [ ]:
### We can split up the 319 missing values by KNN methods, but because it is a neglecting number compare to the 95000 rows, lets drop it.
drop_columns_list +=['GEOCODE2']

## WEALTH1

In [ ]:
numerical['WEALTH1'].isna().sum()

In [ ]:
np.mean(numerical["WEALTH1"])

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,5))
sns.histplot(numerical['WEALTH1'], ax = ax[0])
sns.histplot(numerical['WEALTH1'].fillna(np.mean(numerical['WEALTH1'])),ax = ax[1])
plt.show()

## KNN Model

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
numerical[["WEALTH1", "INCOME"]]

In [ ]:
imputer = KNNImputer(n_neighbors=3)
X = imputer.fit_transform(numerical[["WEALTH1", "INCOME", 'ADI', 'DMA']])

In [ ]:
numerical2 = pd.DataFrame(X)
numerical2.columns = ["WEALTH1", "INCOME", 'ADI', 'DMA']

In [ ]:
numerical2 = numerical2.astype(int)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,5))
sns.histplot(numerical2['WEALTH1'], ax = ax[0])
sns.histplot(numerical2['WEALTH1'].fillna(np.mean(numerical2['WEALTH1'])),ax = ax[1])
plt.show()

#### all missing values for Wealth1 is spread out in the previous interger values thanks to KNN smother distribution
#### We add 41 000 missing value out of 92 000, so we couldn't erase them or just decide to convert them to the mean, median or mode : the distribution was nonsense

## ADI

In [ ]:
numerical['ADI'].isna().sum()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,5))
sns.histplot(numerical['ADI'], ax = ax[0])
sns.histplot(numerical['ADI'].fillna(np.median(numerical['ADI'])),ax = ax[1])
plt.show()

In [ ]:
### We can split up the 319 missing values by KNN methods, but because it is a neglecting number compare to the 95000 rows, there is no risk to give the median as value.
numerical["ADI"].fillna(np.median(numerical['ADI']))

## DMA

In [ ]:
numerical['DMA'].isna().sum()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,5))
sns.histplot(numerical['DMA'], ax = ax[0])
sns.histplot(numerical["DMA"].fillna(np.median(numerical['DMA'])), ax = ax[1])
plt.show()

In [ ]:
### We can split up the 319 missing values by KNN methods, but because it is a neglecting number compare to the 95000 rows, there is no risk to give the median as value.
numerical["DMA"].fillna(np.median(numerical['DMA']))

## MSA

In [ ]:
numerical['MSA'].isna().sum()


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,5))
sns.histplot(numerical['MSA'], ax = ax[0])
sns.histplot(numerical['MSA'].fillna(np.median(numerical['MSA'])),ax = ax[1])
plt.show()

In [ ]:
### We can split up the 319 missing values by KNN methods, but because it is a neglecting number compare to the 95000 rows, there is no risk to give the median as value.
numerical["MSA"].fillna(np.median(numerical['MSA']))

In [ ]:
# Use appropriate methods to clean the columns 
# categorical: GEOCODE2 (drop rows with NA's)
# numerical: WEALTH1 (drop rows with NA's), ADI, DMA,and MSA.
# Drop rows for which 'GEOCODE2' has an NA

In [ ]:
# ml = numerical[numerical['WEALTH1'].notna()].index.tolist()
# numerical = numerical.loc[ml,:]
# categorical = categorical.loc[ml,:]

#Wrapping up

In [ ]:
num_columns_to_drop = [col for col in drop_columns_list if col in list(numerical.columns)]
#num_columns_to_drop
numerical = numerical.drop(columns=num_columns_to_drop, axis = 1)

In [ ]:
df = pd.DataFrame(numerical.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

In [ ]:
numerical.to_csv('C:/Users/igrav/Desktop/ironhack/Lecture/Week_7/Day1/AM/numerical7_02.csv')
categorical.to_csv('C:/Users/igrav/Desktop/ironhack/Lecture/Week_7/Day1/AM/categorical7_02.csv')

In [ ]:
Y.to_csv('C:/Users/igrav/Desktop/ironhack/Lecture/Week_7/Day1/AM/target7_02.csv')

Pandas can also use pickles. 

In [ ]:
numerical.to_pickle('C:/Users/igrav/Desktop/ironhack/Lecture/Week_7/Day1/AM/numerical7_02.pkl')
categorical.to_pickle('C:/Users/igrav/Desktop/ironhack/Lecture/Week_7/Day1/AM/categorical7_02.pkl')
Y.to_pickle('C:/Users/igrav/Desktop/ironhack/Lecture/Week_7/Day1/AM/target7.02.pkl')